##### Choi: 
    subfolder:crowd; 
    original data: train_m_tree.json, dev_tree.json test_tree.json;
    ask to Manuel (è dispersa tra i meandri del codice di choi/lopez)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/to_zip/"
path_corpus_data = root_dir + "tesi_magistrale/datasets/corpus_data_folder/"
crowd_path = base_dir + "crowd/"

Mounted at /content/drive/


In [ ]:
import json

with open(crowd_path + "train_m_tree.json", 'r') as inp:
    examples_train = [json.loads(l) for l in inp.readlines()]

with open(crowd_path + "test_tree.json", 'r') as inp:
    examples_test = [json.loads(l) for l in inp.readlines()]

In [ ]:
print("il dataset train contiene ",len(examples_train), "elementi")

il dataset train contiene  19980 elementi


In [ ]:
print("il dataset test contiene ",len(examples_test), "elementi")

il dataset test contiene  1998 elementi


Merge the two dataset train e test

In [ ]:
entire_dataset_list = examples_train + examples_test
print(" uniti train e test contengono ", len(entire_dataset_list), "elementi")

 uniti train e test contengono  21978 elementi


In [ ]:
phrases = []
for i in range (0,2):
  if (len(entire_dataset_list[i]['y_str'])>1):
    for k in range(0,len(entire_dataset_list[i]['y_str'])):
      phrase = []
      phrase = entire_dataset_list[i]['left_context_token'] + (("/" + entire_dataset_list[i]['y_str'][k]).upper().split()) + entire_dataset_list[i]['right_context_token']
      phrases.append(phrase)
  else:
    phrase = []
    phrase = entire_dataset_list[i]['left_context_token'] + (("/" + entire_dataset_list[i]['y_str'][0]).upper().split()) + entire_dataset_list[i]['right_context_token']
    phrases.append(phrase)

print(phrases)

[['The', 'British', 'Information', 'Commissioner', "'s", 'Office', 'invites', '/CITIZEN', 'to', 'locate', 'its', 'address', 'using', 'Google', 'Maps', '.'], ['The', 'British', 'Information', 'Commissioner', "'s", 'Office', 'invites', '/PERSON', 'to', 'locate', 'its', 'address', 'using', 'Google', 'Maps', '.'], ['The', 'British', 'Information', 'Commissioner', "'s", 'Office', 'invites', '/USER', 'to', 'locate', 'its', 'address', 'using', 'Google', 'Maps', '.'], ['Tushar', 'Gandhi', 'said', 'the', 'Australian', '-', 'born', 'tycoon', 'would', 'be', 'arrested', 'if', '/CRIMINAL', 'visited', 'Bombay', 'or', 'New', 'Delhi', 'again', '.'], ['Tushar', 'Gandhi', 'said', 'the', 'Australian', '-', 'born', 'tycoon', 'would', 'be', 'arrested', 'if', '/ENTREPRENEUR', 'visited', 'Bombay', 'or', 'New', 'Delhi', 'again', '.'], ['Tushar', 'Gandhi', 'said', 'the', 'Australian', '-', 'born', 'tycoon', 'would', 'be', 'arrested', 'if', '/BUSINESSPERSON', 'visited', 'Bombay', 'or', 'New', 'Delhi', 'again', 

In [11]:
with open(path_corpus_data + "corpus_data_CHOI.txt", 'w') as f:
    for s in phrases:
        f.write(str(s) + '\n')